In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def clean_education(s):
    lookup = {
        "Doctorate": "doctorate",
        "Masters": "masters",
        "Bachelors": "bachelors",
        "Assoc-voc": "associates",
        "Assoc-acdm": "associates",
        "Prof-school": "associates",
        "HS-grad": "highschool",
        "Some-college": "college_dropout",
    }
    return lookup.get(s, "incomplete_highschool")

In [3]:
def clean_marital_status(s):
    married = {"Married-civ-spouse", "Married-AF-spouse"}
    previously_married = {"Divorced", "Separated", "Widowed", "Married-spouse-absent"}
    never_married = {"Never-married"}

    if s in married:
        return "married"
    if s in previously_married:
        return "previously_married"
    if s in never_married:
        return "never_married"

    raise Exception(f"Status {s} is not accounted for!")

In [4]:
def region_combining(x):
    north_america = ["Canada","Cuba","Dominican-Republic","El-Salvador","Guatemala","Haiti","Honduras","Jamaica","Mexico",
                     "Nicaragua","Outlying-US(Guam-USVI-etc)","Puerto-Rico","Trinadad&Tobago","United-States"]
    asia = ["Cambodia","China","Hong","India","Iran","Japan","Laos","Philippines","Taiwan","Thailand","Vietnam"]
    south_america = ["Columbia", "Ecuador", "Peru"]
    europe = ["England","France","Germany","Greece","Holand-Netherlands","Hungary","Ireland","Italy","Poland","Portugal",
              "Scotland","Yugoslavia"]
    other = ["South", "Unknown", "?"]

    if x in north_america:
        return "north_america"
    elif x in asia:
        return "asia"
    elif x in south_america:
        return "south_america"
    elif x in europe:
        return "europe"
    elif x in other:
        return "other"
    else:
        return x

    raise Exception(f"Country {x} is not accounted for!")

In [5]:
df = (
        pd.read_csv('census_data.csv')
        .rename(
            # replace " ", "-", and "." with "" in column names
            columns=lambda col: col.replace(" ", "_").replace("-", "_").replace(".", "")
        )
        .assign(
            # strip white spaces and replace "-" by "_"
            workclass=lambda df: df["workclass"].str.strip().replace('-', '_', regex=True),
            # strip white spaces and remap education column as defined in the clean_education function
            education=lambda df: df["education"].str.strip().map(clean_education), # or .apply(clean_education)
            # strip white spaces and remap marital_status column as defined in the clean_marital_status function
            marital_status=lambda df: df["marital_status"].str.strip().map(clean_marital_status),
            # strip white spaces and replace "-" by "_"
            occupation=lambda df: df["occupation"].str.strip().replace('-', '_', regex=True),
            # strip white spaces
            race=lambda df: df["race"].str.strip(),
            # strip white spaces
            sex=lambda df: df["sex"].str.strip(),
            # strip white spaces and remap native_country column as defined in the region_combining function,
            # save the result in a new column native_continent
            native_continent=lambda df: df["native_country"].str.strip().apply(region_combining),
            # create a new column from_us to indicate where from the USA
            from_us=lambda df: df["native_country"].str.strip() == "United-States",
            # create a new column salary_gt_50k to indicate salary >50k
            salary_gt_50k=lambda df: df["salary"].str.strip() == ">50K",
        )
        .loc[
            # exclusion criteria
            lambda df: (~df["workclass"].isin(["without_pay", "Never_worked"]))
                        & (df["occupation"] != "Armed_Forces")
        ]
        .drop(["native_country", "salary"], axis=1) # drop unwanted columns
        .assign(
            # impute missing values in age with mean age
            age=lambda df: df["age"].fillna('mean'),
            # replace '?' wih 'Unknown' in workclass and occupation columns
            workclass=lambda df: df["workclass"].str.replace(r'\?','Unknown', regex=True),
            occupation=lambda df: df["occupation"].str.replace(r'\?','Unknown', regex=True)
        )
    )

In [6]:
df.isnull().sum()

age                       0
workclass                 0
fnlwgt                    0
education                 0
education_no_of_years     0
marital_status            0
occupation                0
race                      0
sex                       0
capital_gain              0
capital_loss              0
working_hours_per_week    0
native_continent          0
from_us                   0
salary_gt_50k             0
dtype: int64

In [7]:
df.head()

,age,workclass,fnlwgt,education,education_no_of_years,marital_status,occupation,race,sex,capital_gain,capital_loss,working_hours_per_week,native_continent,from_us,salary_gt_50k
0,39.0,State_gov,77516,bachelors,13,never_married,Adm_clerical,White,Male,2174,0,40,north_america,True,False
1,50.0,Self_emp_not_inc,83311,bachelors,13,married,Exec_managerial,White,Male,0,0,13,north_america,True,False
2,38.0,Private,215646,highschool,9,previously_married,Handlers_cleaners,White,Male,0,0,40,north_america,True,False
3,53.0,Private,234721,incomplete_highschool,7,married,Handlers_cleaners,Black,Male,0,0,40,north_america,True,False
4,28.0,Private,338409,bachelors,13,married,Prof_specialty,Black,Female,0,0,40,north_america,False,False


In [8]:
df.occupation.value_counts()

occupation
Prof_specialty       4140
Craft_repair         4099
Exec_managerial      4066
Adm_clerical         3770
Sales                3650
Other_service        3295
Machine_op_inspct    2002
Unknown              1836
Transport_moving     1597
Handlers_cleaners    1370
Farming_fishing       994
Tech_support          928
Protective_serv       649
Priv_house_serv       149
Name: count, dtype: int64